In [150]:
import helpers
from helpers import *

### W4 O(log n)

In [151]:
qreg = QuantumRegister(4)
w4 = QuantumCircuit(qreg)
w4.x(qreg[1])
Bdirect(w4, qreg, 2, 1, 0.5)
Bdirect(w4, qreg, 0, 1, 0.5)
Bdirect(w4, qreg, 3, 2, 0.5)
print(w4)
job = qiskit.execute(w4, Aer.get_backend('statevector_simulator'))
theorical_psi = job.result().get_statevector(w4)
print(theorical_psi)

         ┌─────────────────┐                            ┌───┐┌────────────────┐»
q9_0: |0>┤ U3(-0.7854,0,0) ├────────────────────────────┤ X ├┤ U3(0.7854,0,0) ├»
         └──────┬───┬──────┘                       ┌───┐└─┬─┘└────────────────┘»
q9_1: |0>───────┤ X ├─────────■────────────────────┤ X ├──■────────────────────»
         ┌──────┴───┴──────┐┌─┴─┐┌────────────────┐└─┬─┘                       »
q9_2: |0>┤ U3(-0.7854,0,0) ├┤ X ├┤ U3(0.7854,0,0) ├──■────■────────────────────»
         ├─────────────────┤└───┘└────────────────┘     ┌─┴─┐┌────────────────┐»
q9_3: |0>┤ U3(-0.7854,0,0) ├────────────────────────────┤ X ├┤ U3(0.7854,0,0) ├»
         └─────────────────┘                            └───┘└────────────────┘»
«           
«q9_0: ──■──
«      ┌─┴─┐
«q9_1: ┤ X ├
«      ├───┤
«q9_2: ┤ X ├
«      └─┬─┘
«q9_3: ──■──
«           
[ 0. +0.j -0.5+0.j  0.5+0.j  0. +0.j -0.5+0.j  0. +0.j  0. +0.j  0. +0.j
  0.5+0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j]


In [152]:
tomo_circuits = state_tomography_circuits(w4, qreg)

In [153]:
# job = qiskit.execute(tomo_circuits, least_busy, shots=4000)
jobs = []
circuits_pool = split_list(tomo_circuits, math.ceil(len(tomo_circuits)/27))
for circuits_list in circuits_pool:
    job = qiskit.execute(circuits_list, melbourne, shots=4000)
    jobs.append(job)

calib_circuit, state_labels = mc.complete_meas_cal(qr=qreg)
job_cal = qiskit.execute(calib_circuit, melbourne, shots=4000)

for job in jobs:
    job_monitor(job, monitor_async = True)

job_monitor(job_cal, monitor_async = True)

HTML(value="<p style='font-size:16px;'>Job Status: job is queued </p>")

HTML(value="<p style='font-size:16px;'>Job Status: job is queued </p>")

HTML(value="<p style='font-size:16px;'>Job Status: job is queued </p>")

In [157]:
results = []
for job in jobs:
    results.append(copy.deepcopy(job.result()))

In [158]:
data = []
for result in results:
    data += result.results
grouped_results = results[0]
grouped_results.results = data

In [159]:
fitter = StateTomographyFitter(grouped_results, tomo_circuits)
fitted_rho = fitter.fit()
fidelity = state_fidelity(theorical_psi, fitted_rho)
print(fidelity)

0.37783759758822777


In [160]:
cal_results = job_cal.result()
meas_fitter = mc.CompleteMeasFitter(cal_results, state_labels)
correct_results = meas_fitter.filter.apply(grouped_results)
fitter = StateTomographyFitter(correct_results, tomo_circuits)
fitted_rho = fitter.fit()
fidelity = state_fidelity(theorical_psi, fitted_rho)
print(fidelity)

0.7267080486060048


### W4 O(n)

In [161]:
qreg_lin = QuantumRegister(4)
w4_lin = QuantumCircuit(qreg_lin)
w4_lin.x(qreg_lin[0])
Bdirect(w4_lin, qreg_lin, 1, 0, 1/4)
Bdirect(w4_lin, qreg_lin, 2, 1, 1/3)
Bdirect(w4_lin, qreg_lin, 3, 2, 0.5)
print(w4_lin)
job_lin = qiskit.execute(w4_lin, Aer.get_backend('statevector_simulator'))
theorical_psi_lin = job_lin.result().get_statevector(w4_lin)
print(theorical_psi_lin)

                 ┌───┐                               ┌───┐     »
q11_0: |0>───────┤ X ├──────────■────────────────────┤ X ├─────»
          ┌──────┴───┴──────┐ ┌─┴─┐┌────────────────┐└─┬─┘     »
q11_1: |0>┤ U3(-0.5236,0,0) ├─┤ X ├┤ U3(0.5236,0,0) ├──■────■──»
          ├─────────────────┴┐└───┘└────────────────┘     ┌─┴─┐»
q11_2: |0>┤ U3(-0.61548,0,0) ├────────────────────────────┤ X ├»
          ├─────────────────┬┘                            └───┘»
q11_3: |0>┤ U3(-0.7854,0,0) ├──────────────────────────────────»
          └─────────────────┘                                  »
«                                                           
«q11_0: ────────────────────────────────────────────────────
«                          ┌───┐                            
«q11_1: ───────────────────┤ X ├────────────────────────────
«       ┌─────────────────┐└─┬─┘                       ┌───┐
«q11_2: ┤ U3(0.61548,0,0) ├──■────■────────────────────┤ X ├
«       └─────────────────┘     ┌─┴─┐┌───────────

In [162]:
tomo_circuits_lin = state_tomography_circuits(w4_lin, qreg_lin)

In [163]:
# job = qiskit.execute(tomo_circuits, least_busy, shots=4000)
jobs_lin = []
circuits_pool_lin = split_list(tomo_circuits_lin, math.ceil(len(tomo_circuits_lin)/27))
for circuits_list in circuits_pool_lin:
    job_lin = qiskit.execute(circuits_list, melbourne, shots=4000)
    jobs_lin.append(job_lin)

calib_circuit_lin, state_labels_lin = mc.complete_meas_cal(qr=qreg_lin)
job_cal_lin = qiskit.execute(calib_circuit_lin, melbourne, shots=4000)

for job in jobs_lin:
    job_monitor(job, monitor_async = True)

job_monitor(job_cal_lin, monitor_async=True)

HTML(value="<p style='font-size:16px;'>Job Status: job is queued </p>")

HTML(value="<p style='font-size:16px;'>Job Status: job is queued </p>")

HTML(value="<p style='font-size:16px;'>Job Status: job is queued </p>")

HTML(value="<p style='font-size:16px;'>Job Status: job is queued </p>")

In [164]:
results_lin = []
for job in jobs_lin:
    results_lin.append(copy.deepcopy(job.result()))

In [165]:
data_lin = []
for result in results_lin:
    data_lin += result.results
grouped_results_lin = results_lin[0]
grouped_results_lin.results = data_lin

In [166]:
fitter_lin = StateTomographyFitter(grouped_results_lin, tomo_circuits_lin)
fitted_rho_lin = fitter_lin.fit()
fidelity_lin = state_fidelity(theorical_psi_lin, fitted_rho_lin)
print(fidelity_lin)

0.2981906685101153


In [167]:
cal_results_lin = job_cal_lin.result()
meas_fitter_lin = mc.CompleteMeasFitter(cal_results_lin, state_labels_lin)
correct_results_lin = meas_fitter_lin.filter.apply(grouped_results_lin)
fitter_lin = StateTomographyFitter(correct_results_lin, tomo_circuits_lin)
fitted_rho_lin = fitter_lin.fit()
fidelity_lin = state_fidelity(theorical_psi_lin, fitted_rho_lin)
print(fidelity_lin)

0.5068284966187133
